In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import cv2
from PIL import Image

class FaceCNN(nn.Module):
    def __init__(self, num_classes=2, img_height=170, img_width=250):
        super(FaceCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.5)

        with torch.no_grad():
            dummy = torch.zeros(1, 3, img_height, img_width)  
            out = self._forward_convs(dummy)
            n_features = out.view(1, -1).size(1)

        self.fc1 = nn.Linear(n_features, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def _forward_convs(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        return x

    def forward(self, x):
        x = self._forward_convs(x)
        x = x.view(x.size(0), -1)     # flatten
        x = self.dropout(F.relu(self.fc1(x)))
        return self.fc2(x)  



In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = FaceCNN(num_classes=2, img_height=170, img_width=220).to(device)
weights_path = r"drive\best_f1_model_faceCNN.pth"
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()


IMG_HEIGHT, IMG_WIDTH = 170, 220
BATCH_SIZE = 8



val_tfms = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


class_names = ["Not Dominik", "Dominik"]



In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import cv2
from PIL import Image
from deepface import DeepFace

In [ ]:

cap = cv2.VideoCapture(0)
frame_count = 0
last_faces = []  

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    if frame_count % 5 == 0:
        try:
            last_faces = DeepFace.extract_faces(
                frame,
                detector_backend='mtcnn',
                enforce_detection=False
            )
        except:
            last_faces = []

    faces = last_faces  

    for face_data in faces:
        facial_area = face_data["facial_area"]
        x, y, w, h = facial_area["x"], facial_area["y"], facial_area["w"], facial_area["h"]

        face_crop = frame[y:y+h, x:x+w]
        if face_crop.size == 0:
            continue

        img_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
        img_tensor = val_tfms(img_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(img_tensor)
            _, predicted = torch.max(outputs, 1)
            label = str(_)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Webcam Face Recognition", frame)

    # Quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
